## Experiments: Comparing RSAST with other methods

In [94]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
from sktime.transformations.panel.shapelet_transform import ShapeletTransform
import matplotlib.pyplot as plt
import os

hyperparameter='ACF&PACF: n_random_points=100 nb_inst_per_class=50'

### Reading Datasets

In [95]:
# Set directory where the csv files are located
directory = '/home/nirojasvar/random_sast/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]

# Set directory where the csv files are located
directory = '/home/nirojasvar/random_sast/ResultsRsast'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.columns

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

In [96]:
df_result=df_result[['accuracy','dataset_name','classifier_name','time']]

df_result=df_result[df_result["classifier_name"]==hyperparameter]
df_result.dataset_name.values


array(['SmoothSubspace', 'SonyAIBORobotSurface2', 'GunPoint',
       'ItalyPowerDemand', 'ShapeletSim', 'SyntheticControl',
       'SonyAIBORobotSurface1', 'DistalPhalanxOutlineAgeGroup',
       'ItalyPowerDemand', 'Chinatown', 'Coffee', 'BirdChicken',
       'SyntheticControl', 'Chinatown', 'SyntheticControl',
       'SonyAIBORobotSurface2', 'SonyAIBORobotSurface1', 'GunPoint',
       'DistalPhalanxOutlineAgeGroup', 'ShapeletSim',
       'SonyAIBORobotSurface1', 'SmoothSubspace', 'ShapeletSim',
       'Chinatown', 'SmoothSubspace', 'BirdChicken', 'ItalyPowerDemand',
       'SonyAIBORobotSurface2', 'GunPoint', 'BirdChicken'], dtype=object)

In [97]:

df_other_methods=df_other_methods.rename(columns={"dataset":"dataset_name","score":"accuracy","method":"classifier_name"})
df_other_methods = df_other_methods[df_other_methods['dataset_name'].isin(df_result.dataset_name.values)]
df_other_methods.columns

Index(['dataset_name', 'accuracy', 'classifier_name'], dtype='object')

In [98]:
# concatenate the two dataframes
concatenated_df = pd.concat([df_other_methods, df_result])

# reset the index of the concatenated dataframe
concatenated_df = concatenated_df.reset_index(drop=True)


In [99]:
concatenated_df['classifier_name'].unique()

array(['ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF', 'InceptionTime',
       'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet', 'ProximityForest', 'WEASEL',
       'Catch22', 'cBOSS',
       'ACF&PACF: n_random_points=100 nb_inst_per_class=50'], dtype=object)

### Overall analysis

#### Accuracy: RSAST vs Other methods

In [100]:
# create a pivot table with the mean of score by hyperparameter
score_by_method=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name'],columns=['dataset_name'], aggfunc='mean')
score_by_method


dataset_name,BirdChicken,Chinatown,Coffee,DistalPhalanxOutlineAgeGroup,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SonyAIBORobotSurface1,SonyAIBORobotSurface2,SyntheticControl
classifier_name,,,,,,,,,,,
ACF&PACF: n_random_points=100 nb_inst_per_class=50,0.85,0.985423,1.0,0.683453,0.982222,0.952381,0.887037,0.906667,0.809207,0.911508,0.285556
BOSS,0.95,0.766764,1.0,0.733813,1.000000,0.908649,1.000000,0.393333,0.632280,0.859391,0.966667
Catch22,0.90,0.930435,1.0,0.705036,0.946667,0.896016,0.994444,0.866667,0.851913,0.922350,0.970000
HIVE-COTEv1,1.00,0.979592,1.0,0.769784,0.993333,0.962099,1.000000,0.980000,0.753744,0.935992,0.996667
InceptionTime,0.95,0.988338,1.0,0.733813,1.000000,0.965986,0.994444,0.986667,0.878536,0.951731,0.996667
ProximityForest,0.85,0.971014,1.0,0.733813,1.000000,0.965986,0.800000,1.000000,0.836938,0.898216,0.996667
RISE,0.95,0.959184,1.0,0.762590,0.980000,0.953353,0.783333,0.873333,0.821963,0.910808,0.666667
ROCKET,0.90,0.982507,1.0,0.748201,1.000000,0.968902,1.000000,0.980000,0.921797,0.916055,1.000000
ResNet,0.90,0.976812,1.0,0.676259,0.993333,0.965015,0.755556,1.000000,0.956739,0.973767,0.996667


In [101]:
t_score_by_method=score_by_method.transpose()
#t_score_by_method[[hyperparameter,"STC","ROCKET"]]


In [102]:
for method in t_score_by_method.columns.drop(hyperparameter):
    x=t_score_by_method[hyperparameter]
    y=t_score_by_method[method]

    # set the style of the plot
    plt.style.use('seaborn')

    # create the plot
    fig, ax = plt.subplots(figsize=(8, 6))

    # plot the line
    ax.plot([0,1], [0,1])

    # plot the points
    ax.scatter(x, y, color='blue', edgecolor='blue', s=30, zorder=10)

    # add labels and title
    ax.set_xlabel('RSAST')
    ax.set_ylabel(method)
    ax.set_title('Accuracy')

    # set the background color
    ax.set_facecolor('lightgrey')
    plt.savefig('RSASTvs'+method+'.png')


/tmp/ipykernel_2904/2076115059.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')
/tmp/ipykernel_2904/2076115059.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')
/tmp/ipykernel_2904/2076115059.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.s

In [103]:
# create a pivot table with the mean of time by hyperparameter
time_per_method=pd.pivot_table(df_result, values='time', columns=['classifier_name'],index=['dataset_name'], aggfunc='mean')/60
time_per_method

classifier_name,ACF&PACF: n_random_points=100 nb_inst_per_class=50
dataset_name,
BirdChicken,13.528151
Chinatown,0.019082
Coffee,5.698254
DistalPhalanxOutlineAgeGroup,24.781057
GunPoint,5.642177
ItalyPowerDemand,0.136053
ShapeletSim,8.205093
SmoothSubspace,0.253635
SonyAIBORobotSurface1,0.116553


In [104]:
# create a pivot table with the variation of score by hyperparameter
pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name'],columns=['dataset_name'], aggfunc='var')


dataset_name,BirdChicken,Chinatown,DistalPhalanxOutlineAgeGroup,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SonyAIBORobotSurface1,SonyAIBORobotSurface2,SyntheticControl
classifier_name,,,,,,,,,,
ACF&PACF: n_random_points=100 nb_inst_per_class=50,0.0,0.0,0.0,0.000015,0.000007,0.00001,0.000178,0.000034,0.000005,0.001737


#### Generate CD Diagram comparison

In [105]:
from cd_diagram.cd_function import *
df_perf=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
draw_cd_diagram(df_perf, labels=True, title="Overall Comparison Other Methods", name_file='cd-diagram_overall_comparison_other_method.png')


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [106]:
df_perf["classifier_name"].unique()

array(['ACF&PACF: n_random_points=100 nb_inst_per_class=50', 'BOSS',
       'Catch22', 'HIVE-COTEv1', 'InceptionTime', 'ProximityForest',
       'RISE', 'ROCKET', 'ResNet', 'S-BOSS', 'STC', 'TS-CHIEF', 'TSF',
       'WEASEL', 'cBOSS'], dtype=object)

In [107]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,0
BOSS,10.409091
Catch22,10.409091
ACF&PACF: n_random_points=100 nb_inst_per_class=50,10.227273
S-BOSS,10.227273
cBOSS,9.590909
RISE,9.500000
TSF,9.409091
ProximityForest,7.500000
ResNet,7.045455
WEASEL,6.954545
